In [1]:
import time
import datetime
from yahoo_fin import stock_info as yf

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [2]:
def create_dataframe(asset, years_back, interval='1d'):
    return yf.get_data(
            ticker=asset+'=X', 
            start_date=(datetime.date.today() - datetime.timedelta(days=int(years_back*366))).strftime('%Y-%m-%d'), 
            end_date=time.strftime('%Y-%m-%d'), 
            interval=interval
        )

In [3]:
df = create_dataframe(asset='USDKES', years_back=3)
df.tail()

,open,high,low,close,adjclose,volume,ticker
2024-08-08,128.500000,129.139999,128.000000,128.500000,128.500000,0.0,USDKES=X
2024-08-11,128.647308,128.654388,127.750000,128.250000,128.250000,0.0,USDKES=X
2024-08-12,128.000000,128.009995,127.750000,128.000000,128.000000,0.0,USDKES=X
2024-08-13,128.000000,128.250000,127.750000,128.000000,128.000000,0.0,USDKES=X
2024-08-15,128.479996,128.479996,128.479996,128.479996,128.479996,0.0,USDKES=X


In [4]:
df['date'] = df.index 
df = df[['close', 'date']]
df.tail() 

,close,date
2024-08-08,128.500000,2024-08-08
2024-08-11,128.250000,2024-08-11
2024-08-12,128.000000,2024-08-12
2024-08-13,128.000000,2024-08-13
2024-08-15,128.479996,2024-08-15


In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['close'] = scaler.fit_transform(df[['close']])
df.tail()

,close,date
2024-08-08,0.370438,2024-08-08
2024-08-11,0.365876,2024-08-11
2024-08-12,0.361314,2024-08-12
2024-08-13,0.361314,2024-08-13
2024-08-15,0.370073,2024-08-15


In [6]:
import numpy as np

In [7]:
days = 3

df['future'] = df['close'].shift(-days)
last_sequence = np.array(df[['close']].tail(days))
df = df.dropna()
df.tail()

,close,date,future
2024-08-05,0.379562,2024-08-05,0.370438
2024-08-06,0.370438,2024-08-06,0.365876
2024-08-07,0.361314,2024-08-07,0.361314
2024-08-08,0.370438,2024-08-08,0.361314
2024-08-11,0.365876,2024-08-11,0.370073


In [8]:
from collections import deque

In [9]:
N_STEPS = 7

sequence_data = []
sequences = deque(maxlen=N_STEPS)

for entry, target in zip(df[['close', 'date']].values, df['future'].values):
    sequences.append(entry)
    if len(sequences) == N_STEPS:
        sequence_data.append([np.array(sequences), target])

In [10]:
last_sequence = list([s[:len(['close'])] for s in sequences]) + list(last_sequence)
last_sequence = np.array(last_sequence).astype(np.float32)
last_sequence[-7:]

array([[0.37043798],
       [0.3613139 ],
       [0.37043798],
       [0.36587596],
       [0.3613139 ],
       [0.3613139 ],
       [0.37007296]], dtype=float32)

In [11]:
X, y = [], []
for seq, target in sequence_data:
    X.append(seq)
    y.append(target)

features = np.array(X)
target = np.array(y)

target[-7:]

array([0.37043799, 0.3613139 , 0.37043799, 0.36587595, 0.3613139 ,
       0.3613139 , 0.37007295])

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
features

array([[[0.0, Timestamp('2021-08-12 00:00:00')],
        [0.0018249287946421333, Timestamp('2021-08-15 00:00:00')],
        [0.0036497183667638655, Timestamp('2021-08-16 00:00:00')],
        ...,
        [0.00547450793888582, Timestamp('2021-08-18 00:00:00')],
        [0.005657028662854113, Timestamp('2021-08-19 00:00:00')],
        [0.007299297511007774, Timestamp('2021-08-22 00:00:00')]],

       [[0.0018249287946421333, Timestamp('2021-08-15 00:00:00')],
        [0.0036497183667638655, Timestamp('2021-08-16 00:00:00')],
        [0.004562043541564753, Timestamp('2021-08-17 00:00:00')],
        ...,
        [0.005657028662854113, Timestamp('2021-08-19 00:00:00')],
        [0.007299297511007774, Timestamp('2021-08-22 00:00:00')],
        [0.008211761908328619, Timestamp('2021-08-23 00:00:00')]],

       [[0.0036497183667638655, Timestamp('2021-08-16 00:00:00')],
        [0.004562043541564753, Timestamp('2021-08-17 00:00:00')],
        [0.00547450793888582, Timestamp('2021-08-18 00:00:0

In [14]:
features = features[:, :, :len(['close'])].astype(np.float32)
features

array([[[0.        ],
        [0.00182493],
        [0.00364972],
        ...,
        [0.00547451],
        [0.00565703],
        [0.0072993 ]],

       [[0.00182493],
        [0.00364972],
        [0.00456204],
        ...,
        [0.00565703],
        [0.0072993 ],
        [0.00821176]],

       [[0.00364972],
        [0.00456204],
        [0.00547451],
        ...,
        [0.0072993 ],
        [0.00821176],
        [0.0072993 ]],

       ...,

       [[0.37043798],
        [0.3613139 ],
        [0.37956208],
        ...,
        [0.37956208],
        [0.37043798],
        [0.3613139 ]],

       [[0.3613139 ],
        [0.37956208],
        [0.35821176],
        ...,
        [0.37043798],
        [0.3613139 ],
        [0.37043798]],

       [[0.37956208],
        [0.35821176],
        [0.37956208],
        ...,
        [0.3613139 ],
        [0.37043798],
        [0.36587596]]], dtype=float32)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=3)

In [16]:
import tensorflow

In [17]:
model = tensorflow.keras.Sequential([
  tensorflow.keras.layers.LSTM(60, return_sequences=True),
  tensorflow.keras.layers.Dropout(0.3),
  tensorflow.keras.layers.LSTM(120, return_sequences=False),
  tensorflow.keras.layers.Dropout(0.3),
  tensorflow.keras.layers.Dense(20),
  tensorflow.keras.layers.Dense(1),
])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(features, target, epochs=100);

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0010 - loss: 0.0762    
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0046 - loss: 0.0043
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0028 - loss: 0.0028 
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0017 - loss: 0.0033
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0015 - loss: 0.0030
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0010 - loss: 0.0025    
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 9.0797e-04 - loss: 0.0030
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 9.0797e-04 - loss: 0.0025
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0024 - loss: 0.0022
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 4.3694e-04 - loss: 0.0030
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0021 - loss: 0.0019
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
last_sequence = last_sequence[-N_STEPS:]
prediction = model.predict(last_sequence)
predicted_price = scaler.inverse_transform(prediction)[0][0]
predicted_price

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step


128.41455